In [28]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import urllib, json

df_novo = np.load('resultado.npy', allow_pickle = True)

In [29]:
df_novo = pd.DataFrame(df_novo)
colunas = ['parent_mass_cacau', 'espectro_cacau', 'indice_base_atual',
       'espectro_base_real', 'parent_mass_base', 'ms2_cacau',
       'intensity_cacau', 'ms2_base', 'intensity_base', 'similaridade']
for i, j in enumerate(colunas):
    df_novo.rename(columns = {i: j}, inplace=True)
df_novo

,parent_mass_cacau,espectro_cacau,indice_base_atual,espectro_base_real,parent_mass_base,ms2_cacau,intensity_cacau,ms2_base,intensity_base,similaridade
0,194.080875,0,2017,2923,194.080376,"[110.07, 138.07]","[0.5, 1.0]","[61.01, 62.01, 63.01, 69.05, 70.96, 75.95, 89....","[0.11, 0.01, 0.02, 0.01, 0.01, 0.01, 0.01, 0.0...",6.25


In [30]:
massas = []
for i in range(len(df_novo)):
    a = df_novo['ms2_cacau'][i]
    massas.append(a)

In [31]:
prep = pd.DataFrame(massas).T
prep['source'] = 0
prep.rename(columns = {0: 'target'}, inplace=True)
prep['target'] = prep['target'].astype(str)
prep['weight'] = prep['target'].astype(float)

massas = []
for i in range(len(df_novo)):
    a = df_novo['ms2_base'][i]
    massas.append(a)

prep_base = pd.DataFrame(massas).T
prep_base['source'] = 1
prep_base.rename(columns = {0: 'target'}, inplace=True)
prep_base['target'] = prep_base['target'].astype(str)
prep_base['weight'] = prep_base['target'].astype(float)

df_final = prep.append(prep_base)


massas = []
massas.append('Sample')
massas.append('Base')

inicio = pd.DataFrame(massas)

inicio['source']  = -1

inicio.rename(columns = {0: 'target'}, inplace=True)

inicio['weight'] = 100

df_f = inicio.append(df_final)
df_f.reset_index(inplace=True, drop=True)

insercao = df_f.loc[df_f['source']==0]

mais = insercao.index

df_f.reset_index(inplace=True)
indi = df_f['index'][2:].values
sources = df_f['source'][2:].values

verificar = []
for i in indi:
    verificar.append(i)
for i in mais:
    verificar.append(i)
    
source_sem = []
for i in sources:
    source_sem.append(i)
for i in range(len(insercao)):
    source_sem.append(1)
    
# data
label = df_f['target'].values
source = source_sem
target = verificar
value = df_f['weight'].values
# data to dict, dict to sankey
link = dict(source = source, target = target, value = value)
node = dict(label = label, pad=50, thickness=5)
data = go.Sankey(link = link, node=node)
# plot
fig = go.Figure(data)
fig.update_layout(title_text= f"Molecular Match for {df_novo['parent_mass_cacau'].values} with{df_novo['similaridade'].values} similarity",
                  font_size=12)
fig.show()
